In [3]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import os 
import re
import requests
import numpy as np
from collections import Counter
from math import nan
from matplotlib import pyplot as plt
from statsmodels import api as sm
from sys import exit, stderr
from scipy.stats import linregress

# Database

## Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [4]:
# Connect to SQLite database in Python 
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [5]:
path = '../FinalData/SFbikestations_POI_sql_database.sqlite'
connection = create_connection(path)

Connection to SQLite DB successful


In [6]:
# This code defines a function called "execute_query" that allows for executing SQL queries in SQLite
# It uses the cursor's .execute() method to execute the query
# The function takes in two arguments: the connection object and a query strin
# The query string is passed to the .execute() method and can contain any type of SQL query, including creating tables
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [7]:
create_data_by_business_table = """
CREATE TABLE IF NOT EXISTS data_by_business(
  ID INTEGER PRIMARY KEY AUTOINCREMENT, 
  POI_Query TEXT NOT NULL, 
  POI_Categories TEXT NOT NULL, 
  POI_Name VARCHAR NOT NULL, 
  POI_Address VARCHAR NOT NULL,
  POI_Mean_Bayesian_Adj_Rating_Norm FLOAT NOT NULL,
  POI_FSQ_Bayesian_Adj_Rating FLOAT,
  POI_FSQ_Rating FLOAT,
  POI_FSQ_Num_Ratings INT, 
  POI_Yelp_Bayesian_Adj_Rating_Norm FLOAT,
  POI_Yelp_Bayesian_Adj_Rating FLOAT, 
  POI_Yelp_Rating FLOAT,
  POI_Yelp_Num_Ratings FLOAT
  
);
"""

execute_query(connection, create_data_by_business_table) 

Query executed successfully


In [16]:
# Populate the new table with values from the .csv file 
DataByBusiness = pd.read_csv("../final_data/DataByBusiness.csv")
DataByBusiness.to_sql("data_by_business", connection, if_exists='replace', index=False)

9327

In [36]:
# Check to see if the SQLite table matches the orginal .csv 
# They both have the same number of rows, and the first and last rows match 
cursor = connection.cursor()
cursor.execute('SELECT * from data_by_business')
 
# Fetch all records as list of tuples
SF_POI_by_business = cursor.fetchall()
 
# Display result 
for row in SF_POI_by_business:
    # show row
    print(row)

('Restaurants', 'French, Italian, Spanish', ' Ct', '5478 College Ave Oakland, CA 94618', 0.6481617025147504, None, None, None, None, 0.6481617025147504, 3.980772951994084, 4.0, 1387.0)
('Restaurants', 'Greek, Mediterranean, Salad', '$5 Salad', '60 Morris St San Francisco, CA 94103', 0.6081364049845138, None, None, None, None, 0.6081364049845138, 3.891520400662619, 2.5, 3.0)
('Restaurants', 'Japanese Curry, Ramen', '$6 Dollar Bowl Shop', '90 Charter Oak Ave San Francisco, CA 94124', 0.6514234763731033, None, None, None, None, 0.6514234763731033, 3.988046392959872, 4.5, 57.0)
('Restaurants', 'Burgers, Fast Food', '1 800 Burgers', '5333 Adeline St Oakland, CA 94608', 0.6012197140711179, None, None, None, None, 0.6012197140711179, 3.87609684733799, 1.5, 4.0)
('Bars', 'Seafood Markets, Wine Bars', '1 Fish 2 Fish Market', '141 Gough St San Francisco, CA 94102', 0.6163831621833562, None, None, None, None, 0.6163831621833562, 3.90990987346171, 5.0, 2.0)
('Funeral Homes', 'Funeral Home', '1 Hou

In [33]:
create_data_by_station_table = """
CREATE TABLE IF NOT EXISTS data_by_station(
  ID INTEGER PRIMARY KEY AUTOINCREMENT, 
  Bike_Station_ID VARCHAR NOT NULL, 
  Bike_Station_Coordinates VARCHAR NOT NULL, 
  POI_API TEXT NOT NULL, 
  POI_Mean_Distance FLOAT NOT NULL,
  POI_FSQ_Distance FLOAT, 
  POI_Yelp_Distance FLOAT,
  POI_Query TEXT NOT NULL,
  POI_Categories TEXT NOT NULL,
  POI_Name VARCHAR NOT NULL, 
  POI_Address VARCHAR NOT NULL,
  POI_Mean_Bayesian_Adj_Rating_Norm FLOAT,
  POI_FSQ_Bayesian_Adj_Rating_Norm FLOAT,
  POI_FSQ_Bayesian_Adj_Rating FLOAT, 
  POI_FSQ_Rating FLOAT,
  POI_Yelp_Bayesian_Adj_Rating_Norm FLOAT,
  POI_Yelp_Bayesian_Adj_Rating FLOAT,
  POI_Yelp_Rating FLOAT,
  POI_Yelp_Num_Ratings FLOAT,
  FOREIGN KEY (POI_Name) REFERENCES data_by_business(POI_Name)
  FOREIGN KEY (POI_Address) REFERENCES data_by_business(POI_Address)
); 
"""

In [39]:
# Populate the new table with values from the .csv file 
DataByStation = pd.read_csv("../final_data/DataByStation.csv")
DataByStation.to_sql("data_by_station", connection, if_exists='replace', index=False)

59329

In [42]:
cursor.execute('SELECT * from data_by_station')
SF_POI_by_station = cursor.fetchmany(10)
for row in SF_POI_by_station:
    print(row)

('003d58af-e449-4800-aa6d-eb1bba8ad68d', '37.726266,-122.479462', 9.0, 'Both', 423.0, 418.0, 428.0, 'Restaurants', 'Sandwich Restaurant, Soup Restaurant, Vietnamese Restaurant', 'Ha Tien Cove', '1650 Holloway Ave, San Francisco, CA 94132', 0.5931111008807091, None, None, None, None, 0.5931111008807091, 3.858015422347776, 3.5, 44.0)
('003d58af-e449-4800-aa6d-eb1bba8ad68d', '37.726266,-122.479462', 9.0, 'FSQ', 297.0, 297.0, None, 'Restaurants', 'Poke Restaurant', 'Poke Bowl', '3251 20th Ave, San Francisco, CA 94132', 0.579709295832914, 0.579709295832914, 8.086415432571, 6.4, 9.0, None, None, None, None)
('003d58af-e449-4800-aa6d-eb1bba8ad68d', '37.726266,-122.479462', 9.0, 'FSQ', 295.0, 295.0, None, 'Restaurants', 'Restaurant', 'Quickly', '3251 20th Ave (Winston Dr), San Francisco, CA 94132', 0.5656261612420743, 0.5656261612420743, 8.048338680764102, 5.6, 9.0, None, None, None, None)
('003d58af-e449-4800-aa6d-eb1bba8ad68d', '37.726266,-122.479462', 9.0, 'FSQ', 295.0, 295.0, None, 'Restau

# Look at the data before and after the join to validate your data.

In [42]:
# This code describes how to retrieve records from an SQLite database
# The cursor's .execute() method is used to run a SELECT query
# After the query is executed, .fetchall() is called to retrieve the results, which are returned as a list of tuples, where each tuple represents a row of data
# To make the process easier, a function called "execute_read_query" is created to simplify this process
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [95]:
# Check if there are any empty Bike_Station_ID in data_by_station table
# If not, return 'none', otherwise, print the rows where Bike_Station_ID is empty 
select_no_bikestation = "SELECT * FROM data_by_station WHERE Bike_Station_ID IS NULL;"

Bike_Station_ID  = execute_read_query(connection, select_no_bikestation)

if not Bike_Station_ID:
    print('none') 
else:
    for station in Bike_Station_ID:
         print(station)

none


In [101]:
# Check if there are any empty Bike_Station_Coordinates in data_by_station table
# If not, return 'none', otherwise, print the rows where Bike_Station_ID is empty 
select_no_bikecoordinates = "SELECT * FROM data_by_station WHERE Bike_Station_Coordinates IS NULL;"

problem_rows  = execute_read_query(connection, select_no_bikecoordinates)

if not problem_rows:
    print('none') 
else:
    for row in problem_rows:
         print(row)

none


In [103]:
# Check if there are any empty POI_API in data_by_station table
# If not, return 'none', otherwise, print the rows where Bike_Station_ID is empty 
select_no_API = "SELECT * FROM data_by_station WHERE POI_API IS NULL;"

problem_rows  = execute_read_query(connection, select_no_API)

if not problem_rows:
    print('none') 
else:
    for row in problem_rows:
         print(station)

none


In [105]:
# Check if there are any empty cells in data_by_business table under columns that shouldn't 
# If not, return 'none', otherwise, print the problematic rows
select_nulls = "SELECT * FROM data_by_business WHERE POI_Query IS NULL UNION ALL SELECT * FROM data_by_business WHERE POI_Categories IS NULL UNION ALL SELECT * FROM data_by_business WHERE POI_Name IS NULL UNION ALL SELECT * FROM data_by_business WHERE POI_Address IS NULL;"

problem_rows  = execute_read_query(connection, select_nulls)

if not problem_rows:
    print('none') 
else:
    for row in problem_rows:
         print(station)

none


In [106]:
# See if the number of rows in data_by_business is the same as DataByBusiness.csv
count_rows = 'SELECT COUNT(*) FROM data_by_business;'
count_rows = execute_read_query(connection, count_rows)
print(count_rows)

[(9327,)]


In [109]:
# The two tables have the same number of rows 
DataByBusiness = pd.read_csv('../final_data/DataByBusiness.csv')
num_rows = len(DataByBusiness)
print(num_rows)

9327


In [110]:
# See if the number of rows in data_by_station is the same as DataByStation.csv
count_rows = 'SELECT COUNT(*) FROM data_by_station;'
count_rows = execute_read_query(connection, count_rows)
print(count_rows)

[(59329,)]


In [112]:
# The two tables have the same number of rows 
DataByBusiness = pd.read_csv('../final_data/DataByStation.csv')
num_rows = len(DataByStation)
print(num_rows)

59329


In [120]:
# This looks for rows where the same business is duplicated in data_by_business
# some restaurant names appear more than once due to several locations, but grouping by name and address should eliminate that issue
duplicate_rows = 'SELECT POI_Name, POI_Address, COUNT(*) FROM data_by_business GROUP BY POI_Name, POI_Address HAVING COUNT(*) > 1;'
problem_rows  = execute_read_query(connection, duplicate_rows)

if not problem_rows:
    print('none') 
else:
    for row in problem_rows:
         print(row)

none


In [50]:
# When grouping POI by stations, the same POI may appear for different stations. Also, the same POI_Name may appear for the same station if it's a chain and there is more than one near a station
# Grouping by Bike_station_ID, POI_Name, POI_Address should eliminate detecting false duplicates 
duplicate_rows = 'SELECT Bike_Station_ID, POI_Name, POI_Address, COUNT(*) FROM data_by_station GROUP BY Bike_station_ID, POI_Name, POI_Address HAVING COUNT(*) > 1;'
problem_rows  = execute_read_query(connection, duplicate_rows)

if not problem_rows:
    print('none') 
else:
    for row in problem_rows:
         print(row)

none
